In [75]:
## import pytorch and related packages

import torch
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import transforms
from torchvision.models import vgg16, VGG16_Weights, efficientnet_b7, EfficientNet_B7_Weights

In [76]:
## import other packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import PIL
import glob
import os
from IPython.display import Image

In [77]:
## setup device for training 

if torch.backends.mps.is_available():    # check if Apple Silicon mps is available
    device = "mps"
elif torch.cuda.is_available():          # check id cuda is available
    device = "cuda"
else:
    device = "cpu"                       # default to cpu if none are available

device

'mps'

In [78]:
## Image processing 

transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])])

In [79]:
## Prepating the data 

train_path = "./Data/train/"
val_path = "./Data/valid/"

train_dataset = datasets.ImageFolder(train_path, transform=transform)
val_dataset = datasets.ImageFolder(val_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

In [80]:
## Declaring the model

model = efficientnet_b7(pretrained=True)

for param in model.parameters():
    param.requires_grad = False


model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Flatten(),
    nn.BatchNorm1d(num_features=2560), 
    nn.Linear(2560, 32),
    nn.BatchNorm1d(32),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(32, 32),
    nn.BatchNorm1d(32),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(32, 32),
    nn.BatchNorm1d(32),
    nn.ReLU(),
    nn.Linear(32, 4),
    nn.Softmax(dim=1)
)

In [81]:
## Set up loss function and optimizer

loss_fn = nn.CrossEntropyLoss().to(device=device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)




lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.10, patience=3, verbose=True, min_lr=1e-7)


# lr_scheduler = ReduceLROnPlateau(optimizer, mode='max', patience=patience,
#                               threshold=threshold, factor=factor,
#                               min_lr=0, verbose=True)
# optimizer = torch.optim.Adam(params=model.fc.parameters(),


# optimizer = torch.optim.AdamW(params=model.fc.parameters(),
#                              lr=0.001,
#                              weight_decay=0.0001)

In [82]:
## Training loop

epochs = 30
model = model.to(device=device)

for epoch in range(epochs):
    train_loss = 0.0
    train_acc = 0

    model.train()

    for (X, y) in train_loader:
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        logits = model(X)
        loss = loss_fn(logits, y)
        _, preds = torch.max(logits, 1)
        train_acc += torch.sum(preds == y).item()

        loss.backward()
        optimizer.step()

        train_loss += loss.item() * y.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    train_acc = train_acc / len(train_loader.dataset)


    val_loss = 0.0    
    val_acc = 0
    
    model.eval()
    with torch.no_grad():
        for (X, y) in val_loader:
            X, y = X.to(device), y.to(device)

            logits = model(X)
            loss = loss_fn(logits, y)

            _, preds = torch.max(logits, 1)
            val_acc += torch.sum(preds == y).item()

            val_loss += loss.item() * y.size(0)
        
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_acc /len(val_loader.dataset)

    lr_scheduler.step(val_loss)
    
    print(f'Epoch {epoch + 1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | 'f'Test Loss: {val_loss:.4f} | Test Acc: {val_acc:.4f}')



Epoch 1/30 | Train Loss: 1.3779 | Train Acc: 0.3051 | Test Loss: 1.3915 | Test Acc: 0.2778
Epoch 2/30 | Train Loss: 1.3179 | Train Acc: 0.4029 | Test Loss: 1.3959 | Test Acc: 0.3333
Epoch 3/30 | Train Loss: 1.2360 | Train Acc: 0.4992 | Test Loss: 1.3439 | Test Acc: 0.4028
Epoch 4/30 | Train Loss: 1.1555 | Train Acc: 0.6036 | Test Loss: 1.3120 | Test Acc: 0.4028
Epoch 5/30 | Train Loss: 1.1614 | Train Acc: 0.5808 | Test Loss: 1.3390 | Test Acc: 0.3611
Epoch 6/30 | Train Loss: 1.1060 | Train Acc: 0.6362 | Test Loss: 1.2913 | Test Acc: 0.4306
Epoch 7/30 | Train Loss: 1.0957 | Train Acc: 0.6411 | Test Loss: 1.2900 | Test Acc: 0.4444
Epoch 8/30 | Train Loss: 1.1085 | Train Acc: 0.6264 | Test Loss: 1.2514 | Test Acc: 0.4861
Epoch 9/30 | Train Loss: 1.0879 | Train Acc: 0.6427 | Test Loss: 1.2596 | Test Acc: 0.4583
Epoch 10/30 | Train Loss: 1.0969 | Train Acc: 0.6460 | Test Loss: 1.2847 | Test Acc: 0.4167
Epoch 11/30 | Train Loss: 1.0839 | Train Acc: 0.6656 | Test Loss: 1.2838 | Test Acc: 0.44